In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import GPy
import sys
sys.path.append('../')
import BayesianOptimization

In [ ]:
# plot histogram of np.random.choice([8, 10, 12], p=[0.25, 0.5, 0.25])
# plot histogram of np.random.choice([8, 10, 12], p=[0.25, 0.5, 0.25], size=1000) with bins=20, normed=True

sample = np.random.choice([8, 10, 12], p=[0.25, 0.5, 0.25], size=1000)

bin_height, bin_boundary = np.histogram(sample,bins=20)
#define width of each column
width = bin_boundary[1]-bin_boundary[0]
#standardize each column by dividing with the maximum height
bin_height = bin_height/float(max(bin_height))
#plot
plt.bar(bin_boundary, np.append(bin_height,0)/2, width = 0.5)
plt.xlim(7.5,12.5)
plt.show()


In [ ]:
# Mixed integer optimization problem
# Objective function 
# f(x,y) = -(10*y1 + 15*y2 + 20*y3 + x1 + 1.5*x2 + 2*x3) + E[- 5*x4(ω) - 9.5*x5(ω) - 0.5*x7(ω) - 0.5*x8(ω) + 25*x9(ω) + 25*x10(ω)]
# where E is the expected value and ω is a random variable
# Constraints
# Equality constraints
# g1(x) = x5(ω) + x6(ω) - (x7(ω) + x8(ω))
# g2(x) = x6(ω) - 0.9*x4(ω)
# g3(x) = x9(ω) - 0.82*x7(ω)
# g4(x) = x10(ω) - 0.95*x8(ω)
# Inequality constraints
# h1(x,y) = y2 + y3 - 1
# h2(x,y) = x1 - U*y1
# h3(x,y) = x2 - U*y2
# h4(x,y) = x3 - U*y3
# h5(x) = x4(ω) - x1
# h6(x) = x7(ω) - x2
# h7(x) = x8(ω) - x3(ω)
# h8(x) = x9(ω) + x10(ω) - d(ω)
# U = 20
# d(ω\_1) = 8, d(ω\_2) = 10, d(ω\_3) = 12
# with probability 0.25, 0.5, 0.25 respectively

# Objective function
def f(x, y):
    return -(10*y[:,0] + 15*y[:,1] + 20*y[:,2] + x[:,0] + 1.5*x[:,1] + 2*x[:,2]) - 5*x[:,3] - 9.5*x[:,4] - 0.5*x[:,6] - 0.5*x[:,7] + 25*x[:,8] + 25*x[:,9]

# Equality constraints
def g(x, y):
    G = np.zeros((x.shape[0], r))
    G[:,0] = x[:,4] + x[:,5] - (x[:,6] + x[:,7])
    G[:,1] = x[:,5] - 0.9*x[:,3]
    G[:,2] = x[:,8] - 0.82*x[:,6]
    G[:,3] = x[:,9] - 0.95*x[:,7]
    return G

# Inequality constraints
def h(x, y, d):
    #d = np.random.choice([8, 10, 12], p=[0.25, 0.5, 0.25])
    H = np.zeros((x.shape[0], s))
    H[:,0] = y[:,1] + y[:,2] - 1
    H[:,1] = x[:,0] - 30*y[:,0]
    H[:,2] = x[:,1] - 30*y[:,1]
    H[:,3] = x[:,2] - 30*y[:,2]
    H[:,4] = x[:,3] - x[:,0]
    H[:,5] = x[:,6] - x[:,1]
    H[:,6] = x[:,7] - x[:,2]
    H[:,7] = x[:,8] + x[:,9] - d
    return H

# Number of continuous variables (n)
n = 10
# Number of discrete variables (m)
m = 3
# Total number of variables (n+m)
n_tot = n + m
# Number of equality constraints (r)
r = 4
# Number of inequality constraints (s)
s = 8
# Bounds

#[11.695906433, 0.0, 10.526315789, 11.695906433, 0.0, 10.526315789, 0.0, 10.526315789, 0.0, 10.0]

x_l = np.zeros(n)
x_u = 20*np.ones(n)
y_l = np.zeros(m, dtype=int)
y_u = np.ones(m, dtype=int)
# Bounds
bounds_x = np.array([x_l, x_u])
bounds_y = np.array([y_l, y_u])

In [ ]:
# Load initial data
D_0 = np.loadtxt("initial_data_ex3.csv", delimiter=",")

# Select surrogate model, AF, kernel and sampling strategy
surrogate = 'SGP'
kernel = GPy.kern.RBF(input_dim=n_tot, variance=1.)
adquisition = "EI"
sample_y = "Bernoulli"
sample_x = "Gaussian"
param_est_Bernoulli = "Bayes"
param_est_Gaussian = "Bayes"

In [ ]:
def noise_h():
    
    return np.random.choice([8, 10, 12], p=[0.25, 0.5, 0.25])

In [ ]:
labels_D = ["x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8","x9", "x10", "y1", "y2", "y3", "Omega"]
labels_D_best = ["x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8","x9", "x10", "y1", "y2", "y3", "Omega", "p", "f"]
points = 100000
for i in range(5):
    # Run the optimization 10 times, save the solutions with tuples
    bo = BayesianOptimization.BO_GPy(f, g, h, None, None, noise_h,
                                    bounds_x, bounds_y, kernel, n, m, 
                                    points = points, t_max = 100, j_max = 50,
                                    surrogate = surrogate, adquisition = adquisition, 
                                    sample_y = sample_y, sample_x = sample_x, param_est_bernoulli = param_est_Bernoulli, param_est_gaussian = param_est_Gaussian,
                                    D_0 = D_0, early_stop=False, DomainReduction=False, verbose=True)
    D, D_best = bo.fit()
    df_D = pd.DataFrame(D, columns=labels_D)
    df_D_best = pd.DataFrame(D_best, columns=labels_D_best)
    df_D.to_csv("Example_3_stoch_final" + ".csv", index=False, mode="a")
    df_D_best.to_csv("Example_3_stoch_best_final" + ".csv", index=False, mode="a")